<a href="https://colab.research.google.com/github/adalbertii/Modele-klasyfikacyjne/blob/main/digit_recognition_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**# Neuronowy model rozpoznawania odręcznie pisanych cyfr**


In [ ]:
!pip install Pillow

In [ ]:
import numpy as np
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.utils import to_categorical
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report,confusion_matrix

sns.set()
from PIL import Image

In [ ]:
# załądowanie danych
# podział na dane treningowe i testowe
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# sprawdzenie rozmiarów stworzonych podzbiorów
X_train.shape, y_train.shape, X_test.shape,y_test.shape

In [ ]:
X_train[0]

In [ ]:
y_train[:10]

In [ ]:
#przegląd przykładowych cyr w zbiorze uczącym
fig = plt.figure()
for i in range(9):
  plt.subplot(3,3,i+1)
  plt.tight_layout()
  plt.imshow(X_train[i], cmap='gray', interpolation='none')
  plt.title("Digit: {}".format(y_train[i]))
  plt.xticks([])
  plt.yticks([])
fig

In [ ]:
# sprawdzenie rozkładu wartości pikseli
fig = plt.figure(figsize=[10,8])
plt.subplot(2,1,1)
plt.imshow(X_train[3], cmap='gray', interpolation='none')
plt.title("Digit: {}".format(y_train[3]))
plt.xticks([])
plt.yticks([])
plt.subplot(2,1,2)
plt.hist(X_train[0].reshape(784))
plt.title("Pixel Value Distribution")
fig

In [ ]:
#sprawdzenie rozkładu unikalnych wartości zmiennej objaśnianej
print(np.unique(y_train, return_counts=True))

In [ ]:
# kształt zbiorów danych przed procesem "reshape" i normalizacją
print("X_train shape", X_train.shape)
print("y_train shape", y_train.shape)
print("X_test shape", X_test.shape)
print("y_test shape", y_test.shape)

In [ ]:
# zbudowanie spłaszconego wektora wejściowego sieci neuronowej z tablic 28x28 pixels (28x28=784)
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
print("shape of X_train:", X_train.shape )
print("shape of X_test:", X_test.shape )

In [ ]:
# przeskalowanie danych wejściowych
X_train /= 255
X_test /= 255

In [ ]:
X_train[0]

In [ ]:
# wyświetlenie kszatłtu danych wejściowych (treniongowych i tesowych)
print("Train matrix shape", X_train.shape)
print("Test matrix shape", X_test.shape)

In [ ]:
print(np.unique(y_train, return_counts=True))

In [ ]:
# zakodować nasze kategorie - cyfry od 0 do 9 - przy użyciu kodowania one-hot.
n_classes = 10
print("Kształt przed kodowaniem 0/1 : ", y_train.shape)

In [ ]:
# kategoryzacja zmiennej objaśnianej
# Wynikiem jest wektor o długości równej liczbie kategorii.
Y_train = to_categorical(y_train, n_classes)
Y_test = to_categorical(y_test, n_classes)
print("Kształt danych treningowych (zmiennej objaśnianej) PO kodowaniu 0/1: ", Y_train.shape)
print("Kształt danych testowych (zmiennej objaśnianej) PO kodowaniu 0/1: ", Y_test.shape)

In [ ]:
# label dla cyfry 5
Y_train[0]

Step 5 - building neural network

In [ ]:
# budowa modelu
# omówić model!!!!
model = Sequential()
model.add(Dense(512, input_shape=(784,))) # input_shape<-----#########
model.add(Activation('relu'))
model.add(Dropout(0.2)) # Dropout<-----#########

model.add(Dense(512))
model.add(Activation('relu')) #relu <-----#########
model.add(Dropout(0.2))

model.add(Dense(10))
model.add(Activation('softmax')) #softmax<-----#########

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, 'model_info.png', show_shapes=True)

In [ ]:
# zapis topologii modelu do struktury JSON
json_m = model.to_json()
json_m

Step 6 - compiling and training model

In [ ]:
# lompilacja modelu
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [ ]:
# trenowanie modelu i zapis metryk procesu uczenia w zmiennej "history'
history = model.fit(X_train, Y_train,
          batch_size=128, epochs=20,
          verbose=1,
          validation_data=(X_test, Y_test))

In [ ]:
# wykresy przebiegu zmiennej 'accuracy, oraz 'losss
fig = plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')

plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

plt.tight_layout()

fig

In [ ]:
#predykcja na danych testowych
predicted_classes = model.predict(X_test)

In [ ]:
np.set_printoptions(precision=5, suppress=True)


In [ ]:
#weryfikacja wyników predykcji dla pojedynczego znaku
#5, 10,20,50

plt.rcParams['figure.figsize'] = (7,14)

figure = plt.figure()
plt.imshow(X_test[88].reshape(28,28), cmap='gray', interpolation='none')
plt.title(
      "Predicted {}, Truth: {}".format(predicted_classes[88],
                                       y_test[88]))

figure


In [ ]:
# predykcja dla całego zbioru testowego
Y_pred = model.predict(X_test)

In [ ]:
Y_pred[5]

In [ ]:
# sformatowanie tablic na potrzeby "confusion_matrix"
Y_pred_classes =np.argmax(Y_pred, axis=1)
Y_true= np.argmax(Y_test, axis=1)

In [ ]:
# Macierz konfuzji
confusion_m= confusion_matrix(Y_true, Y_pred_classes)
print(confusion_m)

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(confusion_m, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f')
plt.title("Heatmap dla predykcji na danych testowych")
plt.savefig("heatmap.png")
plt.show()

In [ ]:
im = Image.open("heatmap.png")
im.show()

In [ ]:
# Raport klasyfikacji
print(classification_report(Y_true, Y_pred_classes))

**Precision** - ile obserwacji przewidywanych jako pozytywne są w rzeczywistości pozytywne

**Recall** - jak wiele obserwacji z wszystkich poytywnych sklasyfikowaliśmy jako pozytywne
